In [ ]:
import math
import numpy as num

In [ ]:
class Person:
  def __init__(self, line_text):
    # integers
    self.xCategorical = [None]*4
    # integers
    self.xBinary = [None]*4
    # floats
    self.xReal = [None]*6

    # label (0 or 1)
    self.y = None
    # Initialize the features
    self.fromString(line_text)

  def fromString(self, line_text):
     #VARIABLES
    self.xBinary[0] = int(line_text[0])
    self.xReal[0] = float(line_text[1])
    self.xReal[1] = float(line_text[2])
    self.xCategorical[0] = int(line_text[3])
    self.xCategorical[1] = int(line_text[4])
    self.xCategorical[2] = int(line_text[5])
    self.xReal[2] = float(line_text[6])
    self.xBinary[1] = int(line_text[7])
    self.xBinary[2] = int(line_text[8])
    self.xReal[3] = float(line_text[9])
    self.xBinary[3] = int(line_text[10])
    self.xCategorical[3] = int(line_text[11])
    self.xReal[4] = float(line_text[12])
    self.xReal[5] = float(line_text[13])

  #CREDIT CARD YES OR NO (1/0)
    self.y = int(line_text[14])

  def __str__(self):
    #VARIABLES
    person_info = "x = (" + \
	    str(self.xBinary[0]) + "; " + \
	    str(self.xReal[0]) + "; " + \
	    str(self.xReal[1]) + "; " + \
	    str(self.xCategorical[0]) + "; " + \
	    str(self.xCategorical[1]) + "; " + \
	    str(self.xCategorical[2]) + "; " + \
	    str(self.xReal[2]) + "; " + \
	    str(self.xBinary[1]) + "; " + \
	    str(self.xBinary[2]) + "; " + \
	    str(self.xReal[3]) + "; " + \
	    str(self.xBinary[3]) + "; " + \
	    str(self.xCategorical[3]) + "; " + \
	    str(self.xReal[4]) + "; " + \
	    str(self.xReal[5]) + \
	    "); y = " + str(self.y) #CREDIT CARD YES/NO (1/0)
    return person_info

The class Data contains a dataset in the format of *credit_train.txt* or *credit_test.txt* (training data and test data) and uses the Person class to represent the customers.  

In [ ]:
class Data():
  def __init__(self, filename):
    self.data = []
    self.loadFile(filename)

  def loadFile(self,filename):
    with open(filename) as file:
      lines = file.readlines()
      for line in lines:
        person = Person(line.split())
        self.data.append(person)
        
  def printList(self):
    for person in self.data:
      print(person)

In [ ]:
"""
Load the train data
"""
trainData = Data("credit_train.txt")
trainData.printList()

In [ ]:
def computeDistance(person1, person2): #we calculated the distance per type of variable (xBinary,xReal,xCategorical)
    """
    This function calculates the distance between two persons from the dataset.
    The datapoints used to describe each person differentiate on three 
    types of numbers. We cannot simply subtract the datapoints of one 
    person from the datapoints of another to calculate the distance between 
    different people. Instead, this function iterates over datapoints (from person1)
    with the same type and subtracts the value of the same datapoint of person2.
    Finally it adds all of these partial distances together to get the total distance 
    between person 1 and person 2.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += ((person1.xReal[i] - person2.xReal[i])**2)
    for i in range(len(person1.xCategorical)):
        distancesq += ((person1.xCategorical[i] - person2.xCategorical[i])**2)
    distance = math.sqrt(distancesq)
    return distance

In [ ]:
"""
Load the test data
"""
testData = Data("credit_test.txt")

In [ ]:
def giveCredit(person, trainData):
    """
    Predicts if a person may recieve a credit card or not based on its closest 
    neighbour from the dataset. To do this, the function compiles a sorted 
    list, consisting of the distance between person1 and all other persons in the database.
    The y-label (credit) of the most nearby person is then set as the y-label of person1.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance(person, trainData.data[i])
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

In [ ]:
giveCredit(testData.data[0], trainData)

In [ ]:
def GetErrorPercentage(trainData, testData):
    """
    Finds the percentage of the test data that is wrongly classified 
    based on the values of the train data. To do this, the function iterates over the test data
    and increases the amount of falsely classified datapoints each time 
    the given credit in the test data does not match with the value giveCredit returns.
    This is used to calculate the percent of wrongly classified data.
    """
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit(testData.data[i], trainData)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent


In [ ]:
from sklearn.metrics import confusion_matrix
preds = [giveCredit(testData.data[i], trainData) for i in range(len(testData.data))]
labels = [testData.data[i].y for i in range(len(testData.data))]
confusion_matrix(labels, preds)



In [ ]:
"""
Prints the percentage of wrongly classified data.
"""
print(GetErrorPercentage(trainData, testData))

In [ ]:
def getRealBounds(data):
    """
    Determines the minimun and maximum for each real valued variable from the data.
    To do this, the function will create a temporary list to store all values of the
    datapoints. Next it will find the highest value and store this in another list 
    called "max". It will do this for the lowest value aswell and store this in a list
    called "min". 


    Then the function will zip these two lists together into a list called bounds
    The function will do this for each "person" in the data set.
    """
    min, max = [], []
    for i in range(len(data.data[0].xReal)):
        temporary = []
        for j in range(len(data.data)):
            temporary.append(data.data[j].xReal[i])
        max.append(max(temporary))
        min.append(min(temporary))
        bounds = list(zip(min,max))
    return bounds


In [ ]:
getRealBounds(trainData)

In [ ]:
def computeDistance2(person1, person2,getRealBounds):
    """
    This function calculates the distance between two people from the dataset.
    Different from the original computeDistance function, computeDistance2 
    calculates the distance using scaled values regarding the datapoints 
    made out of real numbers.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += (((person1.xReal[i] - person2.xReal[i])/(getRealBounds[i][1]-getRealBounds[i][0]))**2)
    for i in range(len(person1.xCategorical)):
        distancesq += ((person1.xCategorical[i] - person2.xCategorical[i]) ** 2)
    distance = math.sqrt(distancesq)
    return distance

In [ ]:

def giveCredit2(person, trainData, getRealBounds):
    """
    The function giveCredit2 predicts if a person may receive a credit card or not based on its closest 
    neighbour from the dataset. The function percentage2 finds the percentage of the 
    test data that is wrongly classified, based on the values of the train data. Different 
    from the original giveCredit- and percentage function, giveCredit2 and percentage2 
    do this by using scaled values regarding the datapoints made out of real numbers.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance2(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

def GetErrorPercentage2(trainData, testData):
    """ Same as GetErrorPercentage but it uses giveCredit2"""
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit2(testData.data[i], trainData, bounds)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent


In [ ]:
print(GetErrorPercentage2(trainData,testData))

In [ ]:

def computeDistance3(person1, person2,getRealBounds):
    """
    This function calculates the distance between two persons from the dataset.
    Different from the original computeDistance2 function, computeDistance3 
    calculates the distance using scaled values regarding the datapoints 
    made out of catagorical variables. To do this, the function assigns a 0 or 1
    when it has been checked if two people from the data set are from the same
    category or not.
    """
    distancesq = 0
    for i in range(len(person1.xBinary)):
        distancesq += ((person1.xBinary[i] - person2.xBinary[i])**2)
    for i in range(len(person1.xReal)):
        distancesq += (((person1.xReal[i] - person2.xReal[i])/(getRealBounds[i][1]-getRealBounds[i][0]))**2)
    for i in range(len(person1.xCategorical)):
        if person1.xCategorical[i] == person2.xCategorical[i]:
            distancesq += 0
        else:
            distancesq += 1
    distance = math.sqrt(distancesq)
    return distance

In [ ]:

def giveCredit3(person, trainData, getRealBounds):
    """
    The function giveCredit3 predictis if a person may receive a credit card or not based on its closest 
    neighbour from the dataset. The function percentage3 finds the percentage of the 
    test data that is wrongly classified based on the values of the train data. Different from the original 
    giveCredit2- and percentage2 function, giveCredit3 and percentage3 do this 
    by using scaled values regarding the datapoints made out of catagorical variables.
    """
    d = []
    for i in range(len(trainData.data)):
        distance = computeDistance3(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    closest_person = d[0]
    label_of_closest_person = closest_person[0]

    credit  = label_of_closest_person
    return bool(credit)

def GetErrorPercentage3(trainData, testData):
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCredit3(testData.data[i], trainData, bounds)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent

In [ ]:
giveCredit3(testData.data[0], trainData, getRealBounds(trainData))

In [ ]:

def giveCreditk(person, trainData, getRealBounds, k):
    """
    Instead of only looking at the neighbour closest to the person we want to predict
    the credit score of, we now want to look at multiple closest neighbours. 
    The function giveCreditk will firstly compile a sorted list with the distance between the person it 
    needs to predict the approval to receive credit for and all of the other people from the dataset. 
    Then the function will take the nearest k neighbours and compute which credit score is the most common. 
    This will be the credit score the function will assign.

    The function percentagek finds the percentage of the test data that is wrongly classified based on the 
    values of the train data when looking at k closest neighbours of the point it will try to classify.
    """
    d = []
    label = []
    for i in range(len(trainData.data)):
        distance = computeDistance3(person, trainData.data[i],getRealBounds)
        d.append( (trainData.data[i].y,distance) )
    d = sorted(d, key = lambda i:i[1])
    k_closest_person = d[0:k]
    for c in k_closest_person:
        label.append(c[0])
    credit = (max(set(label), key = label.count)) #https://stackoverflow.com/questions/20038011/trying-to-find-majority-element-in-a-list
    return bool(credit)

def GetErrorPercentagek(trainData, testData, k):
    
    bounds = getRealBounds(trainData)
    total = len(testData.data)
    false = 0
    for i in range(total):
        credit = giveCreditk(testData.data[i], trainData, bounds, k)
        if credit != bool(testData.data[i].y):
            false += 1
    percent = false/total*100
    return percent

In [ ]:
"""
Calculates the percentage of the test data that is wrongly classified based on the 
values of the traindata with 1, 3 and 5 nearest neighours.
"""
k = 1
while k < 6:
    print(GetErrorPercentagek(trainData,testData,k))
    k += 2

In [ ]:

k = 1
while k < 100:
    print(GetErrorPercentagek(trainData,testData,k))
    k += 2